# **RAG**
Retrieval Augmented Generation

## **1.환경준비**

### (1) 구글 드라이브

#### 1) 구글 드라이브 폴더 생성
* 새 폴더(langchain)를 생성하고
* 제공 받은 파일을 업로드

#### 2) 구글 드라이브 연결

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### (2) 라이브러리

#### 1) 필요한 라이브러리 설치

* requirements.txt 파일의 [경로 복사]를 한 후,
* 아래 경로에 붙여 넣기

In [4]:
# 경로 : /content/drive/MyDrive/langchain/requirements.txt
# 경로가 다른 경우 아래 코드의 경로 부분을 수정하세요.

!pip install -r /content/drive/MyDrive/langchain/requirements.txt

#### 2) 라이브러리 로딩

In [5]:
import pandas as pd
import numpy as np
import os
import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (3) OpenAI API Key 확인

In [6]:
def load_api_key(filepath):
    with open(filepath, 'r') as file:
        return file.readline().strip()

path = '/content/drive/MyDrive/langchain/'

# API 키 로드 및 환경변수 설정
openai.api_key = load_api_key(path + 'api_key.txt')
os.environ['OPENAI_API_KEY'] = openai.api_key

* ⚠️ 아래 코드셀은, 실행해서 key가 제대로 보이는지 확인하고 삭제하세요.

In [ ]:
print(os.environ['OPENAI_API_KEY'])

## **2.Vector DB**

### (1) Chroma DB 구성

* DB 경로 지정
    * 없으면, 새로 폴더를 만들며 DB 생성
    * 있으면, 기존 DB 연결

In [8]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

database = Chroma(persist_directory = path + "db",  # 경로 지정(구글 드라이브에서 db 폴더 생성)
                    embedding_function = embeddings)  # 임베딩 벡터로 만들 모델 지정

* DB 삭제는, 파이썬을 닫고 폴더를 삭제하면 됩니다.

### (2) INSERT

* 두가지 방법
    * (1) 단순 텍스트 입력 : 각 단위 텍스트를 리스트 형태로 입력 **.add_texts()**
    * (2) 텍스트와 메타데이터 입력 : 각 단위 텍스트와 메타정보를 함께 입력 **.add_documents()**

#### 1) .add_texts()

In [9]:
input_list = ['test 데이터 입력1', 'test 데이터 입력2']

# 입력시 인덱스 저장(조회시 사용)
ind = database.add_texts(input_list)
ind

['bea67049-b107-4412-864f-3e5fc38d2a49',
 'eb93b874-7b5d-4709-b311-1b11a28374c5']

In [10]:
database.get(ind)

{'ids': ['bea67049-b107-4412-864f-3e5fc38d2a49',
  'eb93b874-7b5d-4709-b311-1b11a28374c5'],
 'embeddings': None,
 'metadatas': [None, None],
 'documents': ['test 데이터 입력1', 'test 데이터 입력2'],
 'uris': None,
 'data': None}

#### 2) .add_documents()

In [11]:
input_list2 = ['오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.', '어제 주가는 큰 폭으로 상승했습니다.']
metadata = [{'category':'test'}, {'category':'test'}]

doc2 = [Document(page_content = input_list2[i], metadata = metadata[i]) for i in range(2)]
doc2

[Document(page_content='오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.', metadata={'category': 'test'}),
 Document(page_content='어제 주가는 큰 폭으로 상승했습니다.', metadata={'category': 'test'})]

In [12]:
ind2 = database.add_documents(doc2)

In [13]:
database.get(ind2)

{'ids': ['5cbb3582-489d-420a-9e20-7ac991c76401',
  'ddbe9795-d9c4-4727-b6f6-b13121d67917'],
 'embeddings': None,
 'metadatas': [{'category': 'test'}, {'category': 'test'}],
 'documents': ['오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.', '어제 주가는 큰 폭으로 상승했습니다.'],
 'uris': None,
 'data': None}

### (2) 조회

#### 1) 전체 조회

In [14]:
database.get()

{'ids': ['5cbb3582-489d-420a-9e20-7ac991c76401',
  'bea67049-b107-4412-864f-3e5fc38d2a49',
  'ddbe9795-d9c4-4727-b6f6-b13121d67917',
  'eb93b874-7b5d-4709-b311-1b11a28374c5'],
 'embeddings': None,
 'metadatas': [{'category': 'test'}, None, {'category': 'test'}, None],
 'documents': ['오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.',
  'test 데이터 입력1',
  '어제 주가는 큰 폭으로 상승했습니다.',
  'test 데이터 입력2'],
 'uris': None,
 'data': None}

#### 2) 조건조회
* 일반적인 조건 조회를 지원하지 않음.
    * 그래서 전체를 다 불러와서
    * dataframe으로 변환 후 조건 조회

In [15]:
data = database.get()
data = pd.DataFrame(data)
data

,ids,embeddings,metadatas,documents,uris,data
0,5cbb3582-489d-420a-9e20-7ac991c76401,None,{'category': 'test'},오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.,None,None
1,bea67049-b107-4412-864f-3e5fc38d2a49,None,None,test 데이터 입력1,None,None
2,ddbe9795-d9c4-4727-b6f6-b13121d67917,None,{'category': 'test'},어제 주가는 큰 폭으로 상승했습니다.,None,None
3,eb93b874-7b5d-4709-b311-1b11a28374c5,None,None,test 데이터 입력2,None,None


In [16]:
data.loc[data['metadatas'] == {'category': 'test'}]

,ids,embeddings,metadatas,documents,uris,data
0,5cbb3582-489d-420a-9e20-7ac991c76401,None,{'category': 'test'},오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.,None,None
2,ddbe9795-d9c4-4727-b6f6-b13121d67917,None,{'category': 'test'},어제 주가는 큰 폭으로 상승했습니다.,None,None


#### 3) 유사 문서 조회

In [17]:
# 문서 조회1
query = "오늘 낮 기온은?"   # 질문할 문장
k = 3                      # 유사도 상위 k 개 문서 가져오기.

result = database.similarity_search(query, k = k) #← 데이터베이스에서 유사도가 높은 문서를 가져옴
print(result)
print('-'*50)
for doc in result:
    print(f"문서 내용: {doc.page_content}") # 문서 내용 표시

[Document(page_content='오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.', metadata={'category': 'test'}), Document(page_content='어제 주가는 큰 폭으로 상승했습니다.', metadata={'category': 'test'}), Document(page_content='test 데이터 입력1')]
--------------------------------------------------
문서 내용: 오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.
문서 내용: 어제 주가는 큰 폭으로 상승했습니다.
문서 내용: test 데이터 입력1


In [18]:
# 문서 조회2 : 유사도 점수도 함께 조회
query = "오늘 낮 기온은?"   # 질문할 문장
k = 3                      # 유사도 상위 k 개 문서 가져오기.

result = database.similarity_search_with_score(query, k = k) #← 데이터베이스에서 유사도가 높은 문서를 가져옴
print(result)
print('-'*50)
for doc in result:
    print(f"유사도 점수 : {round(doc[1], 5)}, 문서 내용: {doc[0].page_content}") # 문서 내용 표시

[(Document(page_content='오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.', metadata={'category': 'test'}), 0.22557718985895395), (Document(page_content='어제 주가는 큰 폭으로 상승했습니다.', metadata={'category': 'test'}), 0.38111759959777386), (Document(page_content='test 데이터 입력1'), 0.4948085810123292)]
--------------------------------------------------
유사도 점수 : 0.22558, 문서 내용: 오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.
유사도 점수 : 0.38112, 문서 내용: 어제 주가는 큰 폭으로 상승했습니다.
유사도 점수 : 0.49481, 문서 내용: test 데이터 입력1


### (3) 삭제

* 문서의 id 값으로 삭제합니다.

In [19]:
data

,ids,embeddings,metadatas,documents,uris,data
0,5cbb3582-489d-420a-9e20-7ac991c76401,None,{'category': 'test'},오늘 날씨는 매우 맑음. 낮 기온은 30도 입니다.,None,None
1,bea67049-b107-4412-864f-3e5fc38d2a49,None,None,test 데이터 입력1,None,None
2,ddbe9795-d9c4-4727-b6f6-b13121d67917,None,{'category': 'test'},어제 주가는 큰 폭으로 상승했습니다.,None,None
3,eb93b874-7b5d-4709-b311-1b11a28374c5,None,None,test 데이터 입력2,None,None


In [20]:
id_list = list(data.loc[:1,'ids'])
id_list

['5cbb3582-489d-420a-9e20-7ac991c76401',
 'bea67049-b107-4412-864f-3e5fc38d2a49']

In [21]:
database.delete(ids = id_list)

In [22]:
database.get()

{'ids': ['ddbe9795-d9c4-4727-b6f6-b13121d67917',
  'eb93b874-7b5d-4709-b311-1b11a28374c5'],
 'embeddings': None,
 'metadatas': [{'category': 'test'}, None],
 'documents': ['어제 주가는 큰 폭으로 상승했습니다.', 'test 데이터 입력2'],
 'uris': None,
 'data': None}

### 😀실습

* 오늘 날씨와 관련된 document 들을 10건 이상 입력합니다.
* 검색을 통해 유사도 높은 3건의 document 들을 조회합니다.


In [23]:
input_list = ['태풍 종다리가 소멸되며 생긴 큰 비구름대는 북한으로 이동했습니다. 전국 곳곳에 남겨진 구름은 산발적인 비를 내리고 있는데요.',
              '내일 아침 잠시 소강상태를 보였다가 밤이면 대부분 그치겠고, 모레 새벽까지 비가 이어지는 곳도 있겠습니다. 비는 내일 중북부 지역에 80mm 이상 내리겠고, 이 지역에는 시간당 20~30mm의 강한 비가 쏟아질 수 있겠습니다. 또, 중부와 제주에 최대 60mm, 남부에 최대 40mm가 예상됩니다.',
              '돌풍과 벼락이 동반되기도 해 주의가 필요합니다. 내일은 전국에 바람이 초속 15m로 강하겠고, 산지는 초속 20m까지도 불겠습니다. 풍랑특보가 발효된 서해상과 남해상, 제주 해상에는 내일 오후까지 바람이 매우 강하겠고, 물결도 4m까지 높게 일겠습니다.',
              '태풍도 폭염을 이기진 못했습니다. 비가 내리는 중부의 대부분은 폭염주의보, 남부는 폭염 경보 수준의 더위가 이어지고 있습니다. 절기 처서인 내일 서울 30도, 춘천 31도, 대구 35도, 경주 36도, 부산 33도로 중부와 남부의 기온이 사뭇 다르겠습니다.',
              '밤 기온은 여전히 후텁지근하겠습니다. 올해 전국 평균 열대야 일수는 17.8일로, 기상 관측 이래 최장 기록을 매일 세우고 있고요. 서울은 31일째, 제주는 37일째 연속 열대야의 신기록을 경신 중입니다. 이달 말까지도 최저기온이 25도를 웃돌며 열대야는 길어지겠습니다.',
              '다가오는 주말에는 전국에 가끔 구름이 많겠고, 오후에 전국 대부분 지역에 소나기가 내리겠습니다.',
              '전례 없이 길어지는 폭염에 전력 수요도 연일 최고점입니다. 20일 전력 수요는 오후 5시 기준 97.1GW를 넘어섰습니다.',
              '이달 들어서만 여섯번 째 기록 경신입니다. 앞서 19일에는 하루에만 기록이 두 번 깨졌습니다. 고온 다습한 태풍 종다리는 소멸했지만, 남겨진 비구름대 영향으로 당분간 전국에 비가 예보돼 있어, 당국은 전력 수급에 긴장을 놓지 않고 있습니다.',
              '날씨가 흐려지면 태양광 발전량이 줄어드는 만큼, 전력 시장 내 전력 수요가 더 늘어나기 때문입니다. 전력 공급에서 전력 수요를 뺀 여유분인, 공급 예비율은 한때 8.5%까지 떨어졌습니다. 통상 10% 미만으로 떨어지면 정부는 긴장 상태에 돌입하고, 5% 미만이면 비상 대응에 나섭니다.',
              '다만 지난 2011년 블랙아웃 이후 원전 등 추가 설치로 발전량을 크게 키워 당장의 대규모 정전 사태가 발생할 가능성은 적다고 시장은 보고 있습니다. 긴급 전력 수급 상황 점검에 나선 정부는 "송전망의 탄력 운영 등으로 추가 공급 능력을 확보한 상태"라고 설명했습니다.',
              '내일도 해안가에 높은 너울이 밀려들겠습니다. 해수면이 높은 시기여서 안전사고에 더욱 주의해야 합니다. 시설물 점검을 철저히 하시고, 안전사고가 일어나지 않도록 유의하셔야겠습니다.']

# 입력시 인덱스 저장(조회시 사용)
ind = database.add_texts(input_list)
ind

['ae639206-94e2-432c-8fb3-701974bd7899',
 '623fde3a-5b5c-4af6-975d-aee28981df70',
 '84dda504-2318-4053-ad7a-e8365fd0ba9f',
 'b192802e-5d5c-4210-8847-8bd6e1cc0b4e',
 'fd2b10f0-af15-4939-af1f-dc16b251bcad',
 '6a775b04-3f73-4edd-b4e3-9ae5357bb670',
 'edebb028-f78c-432e-a220-407a226f9ae4',
 '45cad4c9-88aa-41b2-8131-da13117afbb2',
 '95b42beb-0715-4d3e-b656-6c0f9d56c425',
 '3cb4915c-b56e-495f-ae8d-50a9d1b2e36c',
 '97ca8e3e-bc8f-410f-ab13-40a8f1bc407f']

In [26]:
query = "언제까지 열대야가 지속돼?"   # 질문할 문장
k = 3                      # 유사도 상위 k 개 문서 가져오기.

result = database.similarity_search_with_score(query, k = k) #← 데이터베이스에서 유사도가 높은 문서를 가져옴
print(result)
print('-'*50)
for doc in result:
    print(f"유사도 점수 : {round(doc[1], 5)}, 문서 내용: {doc[0].page_content}") # 문서 내용 표시

[(Document(page_content='밤 기온은 여전히 후텁지근하겠습니다. 올해 전국 평균 열대야 일수는 17.8일로, 기상 관측 이래 최장 기록을 매일 세우고 있고요. 서울은 31일째, 제주는 37일째 연속 열대야의 신기록을 경신 중입니다. 이달 말까지도 최저기온이 25도를 웃돌며 열대야는 길어지겠습니다.'), 0.33840250623895884), (Document(page_content='이달 들어서만 여섯번 째 기록 경신입니다. 앞서 19일에는 하루에만 기록이 두 번 깨졌습니다. 고온 다습한 태풍 종다리는 소멸했지만, 남겨진 비구름대 영향으로 당분간 전국에 비가 예보돼 있어, 당국은 전력 수급에 긴장을 놓지 않고 있습니다.'), 0.3508421634870353), (Document(page_content='다가오는 주말에는 전국에 가끔 구름이 많겠고, 오후에 전국 대부분 지역에 소나기가 내리겠습니다.'), 0.3771145826206023)]
--------------------------------------------------
유사도 점수 : 0.3384, 문서 내용: 밤 기온은 여전히 후텁지근하겠습니다. 올해 전국 평균 열대야 일수는 17.8일로, 기상 관측 이래 최장 기록을 매일 세우고 있고요. 서울은 31일째, 제주는 37일째 연속 열대야의 신기록을 경신 중입니다. 이달 말까지도 최저기온이 25도를 웃돌며 열대야는 길어지겠습니다.
유사도 점수 : 0.35084, 문서 내용: 이달 들어서만 여섯번 째 기록 경신입니다. 앞서 19일에는 하루에만 기록이 두 번 깨졌습니다. 고온 다습한 태풍 종다리는 소멸했지만, 남겨진 비구름대 영향으로 당분간 전국에 비가 예보돼 있어, 당국은 전력 수급에 긴장을 놓지 않고 있습니다.
유사도 점수 : 0.37711, 문서 내용: 다가오는 주말에는 전국에 가끔 구름이 많겠고, 오후에 전국 대부분 지역에 소나기가 내리겠습니다.


## **3.RAG 구성하기**

### (1) DF to Vector DB

* 샘플데이터 : 오픈소스 생성형 AI에서 주의해야 할 10가지 사항
    * 기사를 csv로 만든 것입니다.

원문보기: https://www.ciokorea.com/news/337152#csidxc7d1d11066fad86a15937e4c3b29c6d



In [27]:
data = pd.read_csv(path + 'sample.csv', encoding='utf-8')
data

,구분,내용
0,낯선 라이선스 약관,현재 오픈소스 AI 라이선스 유형은 다양한 만큼 매우 복잡하다. 오픈소스 AI 모델...
1,낯선 라이선스 약관,메타와 유사하게 애플은 ‘애플 샘플 코드 라이선스’하에 오픈ELM을 출시했다. 이 ...
2,전문성 부족,오픈소스는 스스로 해야 하는 작업이 많을 수 있다. 기업은 코드를 다운로드할 수는 ...
3,전문성 부족,업계 전체에 전문성이 부족하다는 점은 새로운 문제를 야기한다. 원래 오픈소스의 장점...
4,탈옥,LLM 공격 중 유명한 것에 탈옥(jailbreak)이라는 수법이 있다. 탈옥은 의...
5,탈옥,반면에 악의적인 공격자는 접근하기 쉬운 오픈소스 모델을 무료로 다운로드하여 자신의 ...
6,학습 데이터가 가진 위험,"예술가, 작가, 기타 저작권 소유자들이 대형 AI 기업을 상대로 소송을 제기하고 있..."
7,학습 데이터가 가진 위험,노탈의 수기스는 “대형 벤더는 학습 데이터를 구입하고 소송을 벌이는 데 쓸 돈이 있...
8,새로운 보안 위협,생성형 AI 프로젝트는 단순한 코드 그 이상이기 때문에 잠재적으로 보안 위협에 노출...
9,새로운 보안 위협,또 다른 공격 벡터는 모델의 시스템 프롬프트이다. 수기스에 따르면시스템 프롬프트는 ...


In [28]:
# Chroma 데이터베이스 인스턴스 database2 생성
# 폴더를 db2로 만듭니다.

database2 = Chroma(persist_directory = path + "db2", embedding_function = embeddings)

In [29]:
# 데이터프레임의 텍스트 열(시리즈)을 리스트로 변환
text_list = data['내용'].tolist()

# 리스트 내용을 각각 document로 변환
documents = [Document(page_content=text) for text in text_list]

print(documents)

[Document(page_content='현재 오픈소스 AI 라이선스 유형은 다양한 만큼 매우 복잡하다. 오픈소스 AI 모델을 이용하려면 상업적으로 사용해도 괜찮은지, 수정 및 배포가 가능한지, 사내 코드 베이스에 안전하게 통합할 수 있는지 등을 알아봐야 한다. 여기에 몇 가지 새로운 문제가 등장했다. 일단 이전에는 볼 수 없는 제약이 오픈소스 라이선스에 적용됐다.\n메타의 라마(Llama) 라이선스를 예로 보자. 라마는 높은 인기를 자랑하는 오픈소스 LLM이다. 메타는 라마에 대해 ‘개방형 접근권을 제공하고 잠재적인 오용 문제를 해결하기 맞춤형 상용 라이선스 모델’을 적용한다고 밝혔다. 또한 이에 대해 ‘책임 및 보호 조치의 균형을 맞추기 위한 조치’라고 소개하고 있다.\n이런 라이선스 하에 기업은 상업적으로 라마 모델을 사용할 수 있고 개발자는 기본 라마 모델 위에 추가 작업을 만들어 배포할 수 있다. 단 다른 LLM을 개선하기 위해 라마가 출력하는 결과물을 활용할 수는 없다. (라마 파생 모델은 제외된다.) 또한 기업 또는 그 계열사의 월간 사용자가 700명을 초과하는 경우, 메타에게 라이선스 사용 허락을 요청해야 한다. 메타는 이를 승인하거나 승인하지 않을 수 있다. 어떤 기업에서 라마 3를 사용해서 뭔가를 만들었다면, 눈에 잘 띄는 위치에 ‘Built with Llama 3(라마 3를 기반으로 구축했음)’라는 문구를 표기해야 한다.'), Document(page_content='메타와 유사하게 애플은 ‘애플 샘플 코드 라이선스’하에 오픈ELM을 출시했다. 이 라이선스 역시 특허권은 제외하고 저작권 권한만 명시하고 있다. 애플이나 메타 모두 일반적으로 통용되는 오픈소스 라이선스를 사용하지는 않지만, 실제로 코드는 공개되어 있다. 여기에 애플은 실제로 코드뿐만 아니라 모델 가중치, 훈련 데이터 세트, 훈련 로그, 사전 훈련 구성도 공개했다.\n이제 오픈소스 라이선스의 또 다른 측면을 살펴보자. 기존의 오픈소스 소프트웨어에서 핵심은 ‘코드’였다. 그리고 

In [30]:
# Inseert (.add_documents)
database2.add_documents(documents)

['5651ea9a-02d2-40bd-be81-bce86fd36933',
 '9823f784-7b82-4c70-883c-c146dc27eaa7',
 '3cb18132-1120-43f9-905b-2dfe7dd015d8',
 'd9620d74-04bd-415f-8175-8f05d2f09ad5',
 'f86568e0-385a-4af4-878f-5d2079cca7b8',
 '2ba1d7c0-e1bf-4155-ae25-091ca2a5a029',
 'e9eb1388-39c1-4ea8-8f80-753acff4a712',
 '427e6676-a4af-4272-a90d-f7237b4d819e',
 'c3ca6997-b157-4eb3-a0a0-cf6b2a4d8b12',
 '09a5cb09-3de7-4550-8a5b-de2f69f9717a',
 'f24916ab-39a4-44cd-a062-b2c14bd724da',
 '74af2511-7c15-4eb6-a28d-eb60f5038f38',
 'c73d2dfd-c1a3-45ba-bd96-c5dd96d2f6a7']

In [31]:
# 잘 입력되었는지 확인
database2.get()

{'ids': ['09a5cb09-3de7-4550-8a5b-de2f69f9717a',
  '2ba1d7c0-e1bf-4155-ae25-091ca2a5a029',
  '3cb18132-1120-43f9-905b-2dfe7dd015d8',
  '427e6676-a4af-4272-a90d-f7237b4d819e',
  '5651ea9a-02d2-40bd-be81-bce86fd36933',
  '74af2511-7c15-4eb6-a28d-eb60f5038f38',
  '9823f784-7b82-4c70-883c-c146dc27eaa7',
  'c3ca6997-b157-4eb3-a0a0-cf6b2a4d8b12',
  'c73d2dfd-c1a3-45ba-bd96-c5dd96d2f6a7',
  'd9620d74-04bd-415f-8175-8f05d2f09ad5',
  'e9eb1388-39c1-4ea8-8f80-753acff4a712',
  'f24916ab-39a4-44cd-a062-b2c14bd724da',
  'f86568e0-385a-4af4-878f-5d2079cca7b8'],
 'embeddings': None,
 'metadatas': [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 'documents': ['또 다른 공격 벡터는 모델의 시스템 프롬프트이다. 수기스에 따르면시스템 프롬프트는 일반 사용자에게 보통 공개되지 않는다. 시스템 프롬프트는 모델이 원치 않거나 비윤리적인 행동을 인식하는 안전망을 포함하고 있기에 중요하다. 수기스는 “해커가 시스템 프롬프트에 접근해서 모델을 공격하는 방법을 알아낼 수 있다”라고 설명했다.',
  '반면에 악의적인 공격자는 접근하기 쉬운 오픈소스 모델을 무료로 다운로드하여 자신의 환경에서 실행하며 해킹을 시도할 수 있다. 또한 모델이 사용하는 시스템 프롬프트와 안전 기능까지 다 볼 수 

### (2) 질문으로 유사도 높은 문서 조회하기

In [32]:
# 문서 조회 : 유사도 점수도 함께 조회
query = "생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?"   # 질문할 문장
k = 3                      # 유사도 상위 k 개 문서 가져오기.

result = database2.similarity_search_with_score(query, k = k) #← 데이터베이스에서 유사도가 높은 문서를 가져옴
print(result)
print('-'*50)
for doc in result:
    print(f"유사도 점수 : {round(doc[1], 5)}, 문서 내용: {doc[0].page_content}") # 문서 내용 표시

[(Document(page_content='생성형 AI 프로젝트는 단순한 코드 그 이상이기 때문에 잠재적으로 보안 위협에 노출될 수 있는 영역이 더 많다. LLM은 여러 방면에서 악의적인 공격을 받을 수 있다. 수기스에 따르면, 이들은 관리가 제대로 이루어지지 않는 프로젝트의 개발팀에 침투하여 소프트웨어에 바로 악성 코드를 추가할 수 있다. 거기서 끝나지 않고 훈련 데이터, 미세 조정 또는 가중치까지 오염시킬 수 있다.\n수기스는 “해커는 예시 악성 코드로 모델을 재학습시켜 사용자 인프라에 침입할 수 있다”라며 “또는 가짜 뉴스와 잘못된 정보로 모델을 훈련시킬 수도 있다”라고 설명했다.'), 0.2519096048124197), (Document(page_content='사실 많은 사람이 AI 표준에 대해 이야기할 때 윤리, 개인정보 보호, 설명 가능성 등을 이야기한다. 실제로 작년 12월에 발표된 ISO/IEC 42001 표준과 같은 작업이 해당 영역을 다루고 있다. 그리고 지난 4월 미국 국립표준기술연구소(NIST)는 AI 관련 공통 언어를 제시하는 등 AI 표준 계획 초안을 발표했다. 여기에선 주로 위험과 거버넌스 문제에 초점을 맞추고 있다. 그럼에도 기술 표준에 관해서는 아직 정해진 것이 많지 않다.\n클라우드 네이티브 컴퓨팅 재단의 CIO인 테일러 돌잘은 “표준에 대한 논의는 매우 초기 단계에 머물러 있다”라며 ”단 데이터 분류, 학습 데이터, API, 프롬프트에 대한 표준 형식에 대해 의미 있는 논의가 이뤄지고 있다”라고 말했다. 그래도 아직 ‘논의’하는 수준일 뿐이다.'), 0.2645925343300896), (Document(page_content='오픈소스 그룹 일각에서는 AI 모델에 가드레일(허용 가능한 범위를 두는 일종의 가이드라인 또는 도구)을 두는 것 자체를 반대하기도 한다. 모델에 아무런 제한 없어야 더 나은 성능을 발휘할 것이라 믿는 곳도 있다. 기업은 현재 사용하는 오픈소스 모델이 가드레일에 대해 어떤 방향을 추구하는지조차 잘 

### (3) 질문에 대한 답변 받기
* 절차
    * 질문을 받아
    * 유사도 높은 문서를 DB에서 검색(RAG)
    * 질문과 유사도높은 문서로 프롬프트 구성
    * GPT에 질문하고 답변 받기

* RetrievalQA
    * 랭체인에서 제공하는 체인 함수
    * RAG + QA

In [33]:
chat = ChatOpenAI(model="gpt-3.5-turbo")
retriever = database2.as_retriever()
qa = RetrievalQA.from_llm(llm=chat,  retriever=retriever,  return_source_documents=True )

query = "생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?"
result = qa(query)

print(result["result"])

생성형 AI 도입시 예상되는 보안 위협은 다양합니다. 일반적으로, 악의적인 공격자가 프로젝트에 침투하여 악성 코드를 추가하거나 훈련 데이터, 미세 조정, 가중치 등을 오염시키는 등의 위협이 있을 수 있습니다. 또한, 악의적인 공격자는 모델을 재학습시켜 사용자 인프라에 침입하거나 가짜 뉴스와 잘못된 정보를 사용하여 모델을 훈련시키는 등의 방법을 사용할 수도 있습니다. 이러한 공격으로 인해 모델의 성능이 저하되거나 사용자 정보가 노출될 수 있습니다.


* 유사도 높은 문서 4개를 조회하는 것이 기본값임.

In [34]:
result

{'query': '생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?',
 'result': '생성형 AI 도입시 예상되는 보안 위협은 다양합니다. 일반적으로, 악의적인 공격자가 프로젝트에 침투하여 악성 코드를 추가하거나 훈련 데이터, 미세 조정, 가중치 등을 오염시키는 등의 위협이 있을 수 있습니다. 또한, 악의적인 공격자는 모델을 재학습시켜 사용자 인프라에 침입하거나 가짜 뉴스와 잘못된 정보를 사용하여 모델을 훈련시키는 등의 방법을 사용할 수도 있습니다. 이러한 공격으로 인해 모델의 성능이 저하되거나 사용자 정보가 노출될 수 있습니다.',
 'source_documents': [Document(page_content='생성형 AI 프로젝트는 단순한 코드 그 이상이기 때문에 잠재적으로 보안 위협에 노출될 수 있는 영역이 더 많다. LLM은 여러 방면에서 악의적인 공격을 받을 수 있다. 수기스에 따르면, 이들은 관리가 제대로 이루어지지 않는 프로젝트의 개발팀에 침투하여 소프트웨어에 바로 악성 코드를 추가할 수 있다. 거기서 끝나지 않고 훈련 데이터, 미세 조정 또는 가중치까지 오염시킬 수 있다.\n수기스는 “해커는 예시 악성 코드로 모델을 재학습시켜 사용자 인프라에 침입할 수 있다”라며 “또는 가짜 뉴스와 잘못된 정보로 모델을 훈련시킬 수도 있다”라고 설명했다.'),
  Document(page_content='사실 많은 사람이 AI 표준에 대해 이야기할 때 윤리, 개인정보 보호, 설명 가능성 등을 이야기한다. 실제로 작년 12월에 발표된 ISO/IEC 42001 표준과 같은 작업이 해당 영역을 다루고 있다. 그리고 지난 4월 미국 국립표준기술연구소(NIST)는 AI 관련 공통 언어를 제시하는 등 AI 표준 계획 초안을 발표했다. 여기에선 주로 위험과 거버넌스 문제에 초점을 맞추고 있다. 그럼에도 기술 표준에 관해서는 아직 정해진 것이 많지 않다.\n클라우드 네이티브 컴퓨팅 재단의 CIO인 테일러 돌잘은 “표준에 대한 논의는 매우 초기 단계에 머물러

* 유사도 높은 문서 k개로부터 프롬프트 구성하여 질의하기

In [35]:
k = 3
retriever = database2.as_retriever(search_kwargs={"k": k})

qa = RetrievalQA.from_llm(llm=chat,  retriever=retriever,  return_source_documents=True, )

result = qa(query)
result

{'query': '생성형 AI 도입시 예상되는 보안 위협은 어떤 것들이 있어?',
 'result': '생성형 AI 도입시 예상되는 보안 위협은 다양합니다. 일부 위협은 관리가 부실한 프로젝트에 대한 악의적인 침투로 인한 소프트웨어에 악성 코드 추가, 훈련 데이터, 미세 조정, 가중치 오염 등이 있습니다. 또한 해커가 모델을 재학습시켜 사용자 인프라에 침입하거나 가짜 뉴스, 잘못된 정보로 모델을 훈련시키는 등의 위협도 있을 수 있습니다. 또한 오픈소스 생성형 AI 모델의 안전성을 평가하는 독립적인 기관의 부재와 관련된 위험도 있습니다.',
 'source_documents': [Document(page_content='생성형 AI 프로젝트는 단순한 코드 그 이상이기 때문에 잠재적으로 보안 위협에 노출될 수 있는 영역이 더 많다. LLM은 여러 방면에서 악의적인 공격을 받을 수 있다. 수기스에 따르면, 이들은 관리가 제대로 이루어지지 않는 프로젝트의 개발팀에 침투하여 소프트웨어에 바로 악성 코드를 추가할 수 있다. 거기서 끝나지 않고 훈련 데이터, 미세 조정 또는 가중치까지 오염시킬 수 있다.\n수기스는 “해커는 예시 악성 코드로 모델을 재학습시켜 사용자 인프라에 침입할 수 있다”라며 “또는 가짜 뉴스와 잘못된 정보로 모델을 훈련시킬 수도 있다”라고 설명했다.'),
  Document(page_content='사실 많은 사람이 AI 표준에 대해 이야기할 때 윤리, 개인정보 보호, 설명 가능성 등을 이야기한다. 실제로 작년 12월에 발표된 ISO/IEC 42001 표준과 같은 작업이 해당 영역을 다루고 있다. 그리고 지난 4월 미국 국립표준기술연구소(NIST)는 AI 관련 공통 언어를 제시하는 등 AI 표준 계획 초안을 발표했다. 여기에선 주로 위험과 거버넌스 문제에 초점을 맞추고 있다. 그럼에도 기술 표준에 관해서는 아직 정해진 것이 많지 않다.\n클라우드 네이티브 컴퓨팅 재단의 CIO인 테일러 돌잘은 “표준에 대한 논의는 매우 초기 단계에 머물러 있다”라며 

* 실습

In [37]:
k = 3
retriever = database.as_retriever(search_kwargs={"k": k})

qa = RetrievalQA.from_llm(llm=chat,  retriever=retriever,  return_source_documents=True )
query = '열대야가 언제까지 지속돼?'
result = qa(query)
result

{'query': '열대야가 언제까지 지속돼?',
 'result': '열대야가 이번 달 말까지도 계속되어 길어질 것으로 예상됩니다.',
 'source_documents': [Document(page_content='밤 기온은 여전히 후텁지근하겠습니다. 올해 전국 평균 열대야 일수는 17.8일로, 기상 관측 이래 최장 기록을 매일 세우고 있고요. 서울은 31일째, 제주는 37일째 연속 열대야의 신기록을 경신 중입니다. 이달 말까지도 최저기온이 25도를 웃돌며 열대야는 길어지겠습니다.'),
  Document(page_content='이달 들어서만 여섯번 째 기록 경신입니다. 앞서 19일에는 하루에만 기록이 두 번 깨졌습니다. 고온 다습한 태풍 종다리는 소멸했지만, 남겨진 비구름대 영향으로 당분간 전국에 비가 예보돼 있어, 당국은 전력 수급에 긴장을 놓지 않고 있습니다.'),
  Document(page_content='다가오는 주말에는 전국에 가끔 구름이 많겠고, 오후에 전국 대부분 지역에 소나기가 내리겠습니다.')]}